In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/21183-red-dot-prod-desctiption/21183.csv


In [2]:
#Ref: https://www.databricks.com/blog/efficient-fine-tuning-lora-guide-llms

In [3]:
!pip list |grep -i datasets

datasets                                 2.1.0
tensorflow-datasets                      4.9.2


In [4]:
#update datasets to latest, else it'll throw weird error
#ImportError: cannot import name 'SchemaInferenceError' from 'datasets.arrow_writer' 
#(/opt/conda/lib/python3.10/site-packages/datasets/arrow_writer.py)

%pip install -U datasets -qqq

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires pandas<1.6.0dev0,>=1.3, but you have pandas 2.0.3 which is incompatible.
cudf 23.8.0 requires protobuf<5,>=4.21, but you have protobuf 3.20.3 which is incompatible.
cuml 23.8.0 requires dask==2023.7.1, but you have dask 2023.12.1 which is incompatible.
cuml 23.8.0 requires distributed==2023.7.1, but you have distributed 2023.12.1 which is incompatible.
dask-cuda 23.8.0 requires dask==2023.7.1, but you have dask 2023.12.1 which is incompatible.
dask-cuda 23.8.0 requires distributed==2023.7.1, but you have distributed 2023.12.1 which is incompatible.
dask-cuda 23.8.0 requires pandas<1.6.0dev0,

In [5]:
!pip list |grep -i datasets

datasets                                 2.16.1
tensorflow-datasets                      4.9.2


In [6]:
!pip install peft trl -qqq

In [7]:
import pandas as pd
import torch
from datasets import load_dataset
# from datasets import Dataset

In [8]:
#!wget -c https://huggingface.co/datasets/xiyuez/red-dot-design-award-product-description/blob/main/21183.parquet

In [9]:
!ls -l /kaggle/input/21183-red-dot-prod-desctiption/

total 24768
-rw-r--r-- 1 nobody nogroup 25361141 Jan 12 06:48 21183.csv


In [10]:
df = pd.read_csv('/kaggle/input/21183-red-dot-prod-desctiption/21183.csv')
df

,product,category,description,text
0,Biamp Rack Products,Digital Audio Processors,"“High recognition value, uniform aesthetics an...",Product Name: Biamp Rack Products;\n\nProduct ...
1,V33,Video Camera,The V33 livestreaming video camera ensures hig...,Product Name: V33;\n\nProduct Category: Video ...
2,HP LaserJet 5000-6000 and E700-E800 Series MFPs,Multi-Function Printers,The HP LaserJet 5000 to 6000 Series and E700 t...,Product Name: HP LaserJet 5000-6000 and E700-E...
3,Meaco Arete One 20L Dehumidifier,Heating and Air Conditioning Technology,The Meaco Arete One Dehumidifier is characteri...,Product Name: Meaco Arete One 20L Dehumidifier...
4,théATRE Glass Container for Loose Leaf Tea,Food Containers,The design and colouring of the théATRE Glass ...,Product Name: théATRE Glass Container for Loos...
...,...,...,...,...
21193,PPQ 719 B 21 E,Gas Hob,A refined metallic-coloured glass top emphasis...,Product Name: PPQ 719 B 21 E;\n\nProduct Categ...
21194,product,category,description,text
21195,i1Pro,Spectral Measurement Device,"This versatile colour-measurement device, whic...",Product Name: i1Pro;\n\nProduct Category: Spec...
21196,787 Dreamliner,Aircraft Interior,The new 787 Dreamliner combines groundbreaking...,Product Name: 787 Dreamliner;\n\nProduct Categ...


In [11]:
#Load the dataset from the HuggingFace Hub
# url='https://huggingface.co/datasets/xiyuez/red-dot-design-award-product-description/blob/main/21183.parquet'
# rd_ds = load_dataset(url, data_files='21183.parquet')

# from datasets import Dataset

#Load the dataset from the HuggingFace Hub - has issues reading the parquet from url, so manualy uploaded converted CSV

#rd_ds = load_dataset('/wor')

#Convert to pandas dataframe for convenient processing
# rd_df = df


In [12]:
rd_df = df

In [13]:
rd_df[4:7]

,product,category,description,text
4,théATRE Glass Container for Loose Leaf Tea,Food Containers,The design and colouring of the théATRE Glass ...,Product Name: théATRE Glass Container for Loos...
5,LED Hair Regrowth,Light Therapy Device,The adjustable LED Hair Regrowth helmet uses p...,Product Name: LED Hair Regrowth;\n\nProduct Ca...
6,N+1,Bathroom Shelf,"N+1 is a multifunctional, modular bathroom she...",Product Name: N+1;\n\nProduct Category: Bathro...


In [14]:
#Combine the two attributes into an instruction string
rd_df['instruction'] = 'Create a detailed description for the following product: '+ rd_df['product']+', belonging to category: '+ rd_df['category']

rd_df = rd_df[['instruction', 'description']]

#Get a 5000 sample subset for fine-tuning purposes
rd_df_sample = rd_df.sample(n=15000, random_state=42)

In [15]:
#set to display entire content
pd.set_option('display.max_colwidth', None)

In [16]:
rd_df_sample[0:3]

,instruction,description
11648,"Create a detailed description for the following product: smooth, belonging to category: Kitchen Tap","The appearance of the “smooth” kitchen tap is characterised by the natural appearing transition between rectangular and round elements, between hard and soft lines. A pull-out spray head with magnetic holder is integrated discreetly in the tap neck. The spray head is very easy to extract and replace since the frictional resistance has been reduced to a minimum."
8072,"Create a detailed description for the following product: r7.7, belonging to category: Remote Control","The r7.7 was developed with sustainability at the fore. To achieve a carbon-neutral design, the traditional architecture of a remote control needed to be rethought. The number of components and materials was reduced, and no coating, paint or silicone was used. The organic plastic material is safe and easy to recycle, while maintaining the haptic of a premium product."
16658,"Create a detailed description for the following product: Wireless Speaker System, belonging to category: Consumer electronics and cameras","The Wi-Fi speakers enable audio streaming in any room. Advanced audio technology converts lossless digital files to produce genuine hi-fi sound. With elegant enclosures fashioned from real wood and aluminium, along with high-quality audio components and an intuitive user interface, they offer unlimited music enjoyment. The system comprises compact all-in-one speakers and also stereo floorstanding speakers to meet any need."


In [17]:
#rd_df_sample[0:1]['description']

In [18]:
#Define template and format data into the template for supervised fine-tuning
template = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:

{}

### Response:\n"""

# rd_df_sample['prompt'] = rd_df_sample["instruction"].apply(lambda x: template.format(x))
# rd_df_sample.rename(columns={'description': 'response'}, inplace=True)
# rd_df_sample['response'] = rd_df_sample['response'] + "\n### End"
# rd_df_sample = rd_df_sample[['prompt', 'response']]

# rd_df['text'] = rd_df["prompt"] + rd_df["response"]
# rd_df.drop(columns=['prompt', 'response'], inplace=True)

In [19]:
rd_df_sample['prompt']=rd_df_sample["instruction"].apply(lambda x: template.format(x))

In [20]:
rd_df_sample[4:6]

,instruction,description,prompt
18217,"Create a detailed description for the following product: Fiber Modem, belonging to category: Glasfaser-Modem","The fibreglass modem is the interface for the fibreglass network of Deutsche Telekom. The housing combines fibreglass terminal box and modem and thus reduces the space requirement on the wall. It is plain and reserved in design and fits snugly and compactly on the wall. Thanks to a wall mounting plate and a swivel-type cover, the modem is easy and flexible for the technician to install. An LED indicates correct operation.","Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n\nCreate a detailed description for the following product: Fiber Modem, belonging to category: Glasfaser-Modem\n\n### Response:\n"
2361,"Create a detailed description for the following product: COOKI, belonging to category: Multifunctional Cooking Tool","COOKI was developed with the intention of combining multiple kitchen implements into one sustainable product, not least to reduce material consumption. This two-piece kitchen utensil made from coated stainless steel can be used as a fish slice, pasta spoon, serving tongs, etc. Holes integrated into the handle help remove the leaves of coriander and similar herbs from the stem. The kitchen implement is durable and takes up minimal storage space. By reducing the number of kitchen utensils needed, it also saves on the need for recycling.","Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n\nCreate a detailed description for the following product: COOKI, belonging to category: Multifunctional Cooking Tool\n\n### Response:\n"


In [21]:
rd_df_sample.rename(columns={'description': 'response'}, inplace=True)

In [22]:
rd_df_sample['response'] = rd_df_sample['response'] + "\n### End"

In [23]:
rd_df_sample[4:7]

,instruction,response,prompt
18217,"Create a detailed description for the following product: Fiber Modem, belonging to category: Glasfaser-Modem","The fibreglass modem is the interface for the fibreglass network of Deutsche Telekom. The housing combines fibreglass terminal box and modem and thus reduces the space requirement on the wall. It is plain and reserved in design and fits snugly and compactly on the wall. Thanks to a wall mounting plate and a swivel-type cover, the modem is easy and flexible for the technician to install. An LED indicates correct operation.\n### End","Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n\nCreate a detailed description for the following product: Fiber Modem, belonging to category: Glasfaser-Modem\n\n### Response:\n"
2361,"Create a detailed description for the following product: COOKI, belonging to category: Multifunctional Cooking Tool","COOKI was developed with the intention of combining multiple kitchen implements into one sustainable product, not least to reduce material consumption. This two-piece kitchen utensil made from coated stainless steel can be used as a fish slice, pasta spoon, serving tongs, etc. Holes integrated into the handle help remove the leaves of coriander and similar herbs from the stem. The kitchen implement is durable and takes up minimal storage space. By reducing the number of kitchen utensils needed, it also saves on the need for recycling.\n### End","Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n\nCreate a detailed description for the following product: COOKI, belonging to category: Multifunctional Cooking Tool\n\n### Response:\n"
17068,"Create a detailed description for the following product: Mazda6, belonging to category: Passenger Car","Both the sedan and wagon versions of the Mazda6 are generously proportioned, offering outstanding comfort for the driver and passengers. Thanks to groundbreaking technologies developed for the engine, car body and chassis, along with its innovative brake energy regeneration system, it is both powerful and economical. The Mazda6 is surprisingly agile at low speeds and handles with precision at high speeds, while simultaneously delivering a remarkably quiet driving experience.\n### End","Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n\nCreate a detailed description for the following product: Mazda6, belonging to category: Passenger Car\n\n### Response:\n"


In [24]:
rd_df_sample = rd_df_sample[['prompt', 'response']]
rd_df_sample[4:7]

,prompt,response
18217,"Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n\nCreate a detailed description for the following product: Fiber Modem, belonging to category: Glasfaser-Modem\n\n### Response:\n","The fibreglass modem is the interface for the fibreglass network of Deutsche Telekom. The housing combines fibreglass terminal box and modem and thus reduces the space requirement on the wall. It is plain and reserved in design and fits snugly and compactly on the wall. Thanks to a wall mounting plate and a swivel-type cover, the modem is easy and flexible for the technician to install. An LED indicates correct operation.\n### End"
2361,"Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n\nCreate a detailed description for the following product: COOKI, belonging to category: Multifunctional Cooking Tool\n\n### Response:\n","COOKI was developed with the intention of combining multiple kitchen implements into one sustainable product, not least to reduce material consumption. This two-piece kitchen utensil made from coated stainless steel can be used as a fish slice, pasta spoon, serving tongs, etc. Holes integrated into the handle help remove the leaves of coriander and similar herbs from the stem. The kitchen implement is durable and takes up minimal storage space. By reducing the number of kitchen utensils needed, it also saves on the need for recycling.\n### End"
17068,"Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n\nCreate a detailed description for the following product: Mazda6, belonging to category: Passenger Car\n\n### Response:\n","Both the sedan and wagon versions of the Mazda6 are generously proportioned, offering outstanding comfort for the driver and passengers. Thanks to groundbreaking technologies developed for the engine, car body and chassis, along with its innovative brake energy regeneration system, it is both powerful and economical. The Mazda6 is surprisingly agile at low speeds and handles with precision at high speeds, while simultaneously delivering a remarkably quiet driving experience.\n### End"


In [25]:
rd_df_sample['text'] = rd_df_sample["prompt"] + rd_df_sample["response"]

In [26]:
rd_df_sample[4:7]

,prompt,response,text
18217,"Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n\nCreate a detailed description for the following product: Fiber Modem, belonging to category: Glasfaser-Modem\n\n### Response:\n","The fibreglass modem is the interface for the fibreglass network of Deutsche Telekom. The housing combines fibreglass terminal box and modem and thus reduces the space requirement on the wall. It is plain and reserved in design and fits snugly and compactly on the wall. Thanks to a wall mounting plate and a swivel-type cover, the modem is easy and flexible for the technician to install. An LED indicates correct operation.\n### End","Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n\nCreate a detailed description for the following product: Fiber Modem, belonging to category: Glasfaser-Modem\n\n### Response:\nThe fibreglass modem is the interface for the fibreglass network of Deutsche Telekom. The housing combines fibreglass terminal box and modem and thus reduces the space requirement on the wall. It is plain and reserved in design and fits snugly and compactly on the wall. Thanks to a wall mounting plate and a swivel-type cover, the modem is easy and flexible for the technician to install. An LED indicates correct operation.\n### End"
2361,"Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n\nCreate a detailed description for the following product: COOKI, belonging to category: Multifunctional Cooking Tool\n\n### Response:\n","COOKI was developed with the intention of combining multiple kitchen implements into one sustainable product, not least to reduce material consumption. This two-piece kitchen utensil made from coated stainless steel can be used as a fish slice, pasta spoon, serving tongs, etc. Holes integrated into the handle help remove the leaves of coriander and similar herbs from the stem. The kitchen implement is durable and takes up minimal storage space. By reducing the number of kitchen utensils needed, it also saves on the need for recycling.\n### End","Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n\nCreate a detailed description for the following product: COOKI, belonging to category: Multifunctional Cooking Tool\n\n### Response:\nCOOKI was developed with the intention of combining multiple kitchen implements into one sustainable product, not least to reduce material consumption. This two-piece kitchen utensil made from coated stainless steel can be used as a fish slice, pasta spoon, serving tongs, etc. Holes integrated into the handle help remove the leaves of coriander and similar herbs from the stem. The kitchen implement is durable and takes up minimal storage space. By reducing the number of kitchen utensils needed, it also saves on the need for recycling.\n### End"
17068,"Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n\nCreate a detailed description for the following product: Mazda6, belonging to category: Passenger Car\n\n### Response:\n","Both the sedan and wagon versions of the Mazda6 are generously proportioned, offering outstanding comfort for the driver and passengers. Thanks to groundbreaking technologies developed for the engine, car body and chassis, along with its innovative brake energy regeneration system, it is both powerful and economical. The Mazda6 is surprisingly agile at low speeds and handles with precision at high speeds, while simultaneously delivering a remarkably quiet driving experience.\n### End","Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n\nCreate a detailed description for the following product: Mazda6, b

In [27]:
#drop remaining columns
rd_df_sample.drop(columns=['prompt', 'response'], inplace=True)
rd_df_sample[4:7]

,text
18217,"Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n\nCreate a detailed description for the following product: Fiber Modem, belonging to category: Glasfaser-Modem\n\n### Response:\nThe fibreglass modem is the interface for the fibreglass network of Deutsche Telekom. The housing combines fibreglass terminal box and modem and thus reduces the space requirement on the wall. It is plain and reserved in design and fits snugly and compactly on the wall. Thanks to a wall mounting plate and a swivel-type cover, the modem is easy and flexible for the technician to install. An LED indicates correct operation.\n### End"
2361,"Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n\nCreate a detailed description for the following product: COOKI, belonging to category: Multifunctional Cooking Tool\n\n### Response:\nCOOKI was developed with the intention of combining multiple kitchen implements into one sustainable product, not least to reduce material consumption. This two-piece kitchen utensil made from coated stainless steel can be used as a fish slice, pasta spoon, serving tongs, etc. Holes integrated into the handle help remove the leaves of coriander and similar herbs from the stem. The kitchen implement is durable and takes up minimal storage space. By reducing the number of kitchen utensils needed, it also saves on the need for recycling.\n### End"
17068,"Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n\nCreate a detailed description for the following product: Mazda6, belonging to category: Passenger Car\n\n### Response:\nBoth the sedan and wagon versions of the Mazda6 are generously proportioned, offering outstanding comfort for the driver and passengers. Thanks to groundbreaking technologies developed for the engine, car body and chassis, along with its innovative brake energy regeneration system, it is both powerful and economical. The Mazda6 is surprisingly agile at low speeds and handles with precision at high speeds, while simultaneously delivering a remarkably quiet driving experience.\n### End"


In [28]:
!pip install accelerate -qqq
#!pip install -i https://test.pypi.org/simple/ bitsandbytes -qqq
!pip install -U bitsandbytes -qqq    

In [29]:
!pip install flash-attn --no-build-isolation -qqq

In [30]:
from transformers import BitsAndBytesConfig
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,            # load model in 4-bit precision
    bnb_4bit_quant_type="nf4",    # pre-trained model should be quantized in 4-bit NF format
    bnb_4bit_use_double_quant=False, # Using double quantization as mentioned in QLoRA paper
    bnb_4bit_compute_dtype=torch.bfloat16, # During computation, pre-trained model should be loaded in BF16 format
)

In [31]:
import torch
from transformers import LlamaTokenizer, LlamaForCausalLM

model_path = 'openlm-research/open_llama_3b_v2'
tokenizer = LlamaTokenizer.from_pretrained(model_path)
model = LlamaForCausalLM.from_pretrained(
    model_path, 
    quantization_config=bnb_config,
#     load_in_8bit=True,
    device_map='auto',
    #use_flash_attention_2=True  #requires Ampere or Newere GPUs
)

tokenizer_config.json:   0%|          | 0.00/593 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/512k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/330 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/6.85G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [32]:
#Pass in a prompt and infer with the model
prompt = 'Q: Create a detailed description for the following product: Dologic Smooth Mouse, belonging to category: Optical Mouse\nA:'
#prompt='Q: Create a detailed description for the following product: Oral B Battery powerd toothbrush, belonging to category: Battery Toothbrush\nA:'
input_ids = tokenizer(prompt, return_tensors="pt").input_ids

generation_output = model.generate(
input_ids=input_ids, max_new_tokens=128
)

print(tokenizer.decode(generation_output[0]))

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1636: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


<s>Q: Create a detailed description for the following product: Dologic Smooth Mouse, belonging to category: Optical Mouse
A: Dologic Smooth Mouse is a mouse with a smooth surface. It is a wireless mouse with a 2.4 GHz wireless connection. It has a 2.4 GHz wireless connection and a 2.4 GHz wireless connection. It has a 2.4 GHz wireless connection and a 2.4 GHz wireless connection. It has a 2.4 GHz wireless connection and a 2.4 GHz wireless connection. It has a 2.4 GHz wireless connection and a 2.4 GHz wireless connection. It has a 2.4 GHz wireless connection and


In [33]:
prompt= """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Create a detailed description for the following product: Corelogic Smooth Mouse, belonging to category: Optical Mouse

### Response:"""
input_ids = tokenizer(prompt, return_tensors="pt").input_ids

generation_output = model.generate(
input_ids=input_ids, max_new_tokens=128
)

print(tokenizer.decode(generation_output[0]))

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1636: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


<s>Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Create a detailed description for the following product: Corelogic Smooth Mouse, belonging to category: Optical Mouse

### Response:
Corelogic Smooth Mouse is a mouse that is designed to be used by people who have a hard time using a mouse. The mouse is designed to be used by people who have a hard time using a mouse. The mouse is designed to be used by people who have a hard time using a mouse. The mouse is designed to be used by people who have a hard time using a mouse. The mouse is designed to be used by people who have a hard time using a mouse. The mouse is designed to be used by people who have a hard time using a mouse. The mouse is designed to be used by people who have a hard


In [34]:
# import re
# model_modules = str(model.modules)
# pattern = r'\((\w+)\): Linear'
# linear_layer_names = re.findall(pattern, model_modules)

# names = []
# # Print the names of the Linear layers
# for name in linear_layer_names:
#     names.append(name)
# target_modules = list(set(names))

In [35]:
#str(model.modules)

In [36]:
from trl import SFTTrainer
from transformers import TrainingArguments
import time
from datasets import Dataset

In [37]:
#get peft model - not required if you are using SFTTrainer()
#peft_model =get_peft_model(model, lora_config)

In [38]:
# # # #for redoing without restart notebook by stop/start
# del trainer
# del peft_model_saved
# del peft_base_model
# del lora_config

In [39]:
from peft import LoraConfig, get_peft_model


#If only targeting attention blocks of the model
#target_modules = ["q_proj", "v_proj"]

#If targeting all linear layers
target_modules = ['q_proj','k_proj','v_proj','o_proj','gate_proj','down_proj','up_proj','lm_head']

lora_config = LoraConfig(
    r=16,
    target_modules = target_modules,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

In [40]:
output_dir=f'./openllama-3b-red-dot-prod-desc-training-{str(int(time.time()))}'
peft_training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="adamw_hf",   # adamw_apex_fused threw error, apex not found, #paged_adamw_32bit
    num_train_epochs=20,
    save_steps=50, #checkpointing must be high else, disk will get used and train() wll fail
#     eval_steps=25,
    logging_steps=10,
    learning_rate=5e-5,
    weight_decay=0.0001,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=400, # after 1225, leds to error,HDD is full(shown RED):/dev/loop1         20G   20G     0 100% /kaggle/lib
    #-1 led to crash: OSError: [Errno 28] No space left on device, so switch to lesser value say 500 or 1000
    warmup_ratio=0.03,
    group_by_length=True,
    #SchedulerType:['linear', 'cosine', 'cosine_with_restarts', 'polynomial', 'constant', 'constant_with_warmup', 'inverse_sqrt', 'reduce_lr_on_plateau']
    lr_scheduler_type="linear",
    #warmup_steps=200, use  one of w_steps or w_ratio , not both
#     save_total_limit = 2,
#     load_best_model_at_end = True,
#     save_strategy = 'steps',
#     evaluation_strategy = 'steps',
#     eval_steps=25,
    report_to="tensorboard"
)

In [41]:
#!pip list |grep -i torch

In [42]:
#model.add_adapter(lora_config)

In [43]:
trainer = SFTTrainer(
    model,
    train_dataset=Dataset.from_pandas(rd_df_sample[:5000]),
    eval_dataset =Dataset.from_pandas(rd_df_sample[5001:7000]),
    dataset_text_field="text",
    max_seq_length=256,
    args=peft_training_args,
    peft_config=lora_config,
    tokenizer=tokenizer,
    packing=False
)

# Initiate the training process
# with mlflow.start_run(run_name= 'first_run'):

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1999 [00:00<?, ? examples/s]

In [44]:
#Upcast layer norms to float 32 for stability
for name, module in trainer.model.named_modules():
  if "norm" in name:
    module = module.to(torch.float32)

In [45]:
tokenizer.pad_token = tokenizer.eos_token
#tokenizer.add_special_tokens({'pad_token': '[PAD]'})   -

In [46]:
#You are using 8-bit optimizers with a version of `bitsandbytes` < 0.41.1.
# It is recommended to update your version as a major bug has been fixed in 8-bit optimizers.
%time trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
10,2.906100
20,2.510800
30,2.022300
40,1.765300
50,1.715000
60,1.961900
70,1.823200
80,1.727600
90,1.695400
100,1.784600


/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:131: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:131: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:131: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:131: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers fou

CPU times: user 8min 39s, sys: 4min 27s, total: 13min 6s
Wall time: 13min 6s


TrainOutput(global_step=400, training_loss=1.8218593645095824, metrics={'train_runtime': 784.9973, 'train_samples_per_second': 2.038, 'train_steps_per_second': 0.51, 'total_flos': 5378780301726720.0, 'train_loss': 1.8218593645095824, 'epoch': 0.32})

In [47]:
#Save peft model: 
peft_model_path=f"/kaggle/working/openllama-3B-v2-prod-desc-checkpoint-{str(int(time.time()))}"

trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:131: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


('/kaggle/working/openllama-3B-v2-prod-desc-checkpoint-1705285955/tokenizer_config.json',
 '/kaggle/working/openllama-3B-v2-prod-desc-checkpoint-1705285955/special_tokens_map.json',
 '/kaggle/working/openllama-3B-v2-prod-desc-checkpoint-1705285955/tokenizer.model',
 '/kaggle/working/openllama-3B-v2-prod-desc-checkpoint-1705285955/added_tokens.json')

In [48]:
#load for inference:
from peft import PeftModel, PeftConfig

peft_base_model = LlamaForCausalLM.from_pretrained('openlm-research/open_llama_3b_v2', torch_dtype=torch.bfloat16)
tokenizer = LlamaTokenizer.from_pretrained('openlm-research/open_llama_3b_v2')

peft_model_saved = PeftModel.from_pretrained(
    peft_base_model, 
    peft_model_path,
    torch_dtype=torch.bfloat16, 
    is_trainable=False
)

In [54]:
#move to gpu for faster inference
peft_model_saved = peft_model_saved.to('cuda')
#Inference with same input: 
#Pass in a prompt and infer with the model
#prompt = 'Q: Create a detailed description for the following product: Dologic Smooth Mouse, belonging to category: Optical Mouse\nA:'
#prompt=' Q:Create a detailed description for the following product: Oral B Battery powerd toothbrush, belonging to category: Battery Toothbrush\nA:'
#prompt='Q: Create a detailed description for the following product: Portable Car vaccum cleaner, belonging to category: Car Vaccum cleaner\nA:'
prompt='Q: Create a detailed description for the following product: Wireless desktop keyboard, belonging to category: Wireless keyboard\nA:'

input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to('cuda')

generation_output = peft_model_saved.generate(
    input_ids=input_ids, 
    max_new_tokens=200, 
    repetition_penalty=1.3,
    temperature=.3,
    #do_sample=True
)

print(tokenizer.decode(generation_output[0]))

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


<s> Q: Create a detailed description for the following product: Wireless desktop keyboard, belonging to category: Wireless keyboard
A: The wireless mouse and its accompanying USB receiver are hidden in an integrated compartment. This allows users of this device not only freedom from cables but also saves space on their desktops or work surfaces as well – especially when used with laptops that have no built-in ports at all (e.g., Ultrabooks). A special feature is provided by two separate buttons which can be configured individually via software so they perform different functions depending upon how you press them; e. g.: one button opens up your favourite web browser while another activates media playback controls without having any needlessly cluttering other keys around it..</s>


In [52]:
from huggingface_hub import notebook_login

notebook_login()

In [53]:
model.push_to_hub("dolo650/open_llama_3b_v2-qlora-instruct", use_auth_token=True)

/opt/conda/lib/python3.10/site-packages/transformers/utils/hub.py:821: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


NotImplementedError: You are calling `save_pretrained` on a 4-bit converted model. This is currently not supported

In [ ]:
# #Use generation cnfig to tune temp, top_k etc params: 
#Snip: https://github.com/TimDettmers/bitsandbytes/pull/753
# generation_config = GenerationConfig(
#     temperature=0.1,
#     top_p=0.8,
#     top_k=40,
#     # penalty_alpha=0.6,
#     # num_beams=BEAMS,
#     repetition_penalty=1.5,
#     do_sample=True,
#     eos_token_id=tokenizer.eos_token_id,
#     pad_token=tokenizer.pad_token_id,
#     max_length=CTX_LEN,
#     use_cache=True
# )

# print(f'{len(input_ids.squeeze())} tokens in prompt\n\n')

# with torch.no_grad():
#     generation_output = peft_model.generate(
#         input_ids=input_ids,
#         pad_token_id=tokenizer.eos_token_id,
#         attention_mask=attention_mask,
#         generation_config=generation_config,
#         return_dict_in_generate=True,
#         output_scores=True,
#         early_stopping=BEAMS > 1,
#         streamer=streamer
#     )